### Modules used in this notebook
xarray, cfgrib, cartopy, numpy

# C.004: Creating more complex maps

In [ ]:
import numpy as np
import xarray as xr # xarray library access and manipulate multi-dimensional data (such as netcdf). With the cfgrib engine we can manipulate grib files
from matplotlib import pyplot as plt # matplotlib package provides different functions to plot data
import cartopy.crs as ccrs # cartopy package can be integrated with matplotlib to plot georeferenced data
import cartopy.feature

We use xarray to open the dataset, compute the wind speed and the daily mean from hourly data

In [ ]:
ds = xr.open_dataset('../data/era5-u100_v100_201903.grib')
# ds = ds.assign({'ws100': (ds['u100']**2 + ds['v100']**2)**0.5})
ds['ws100'] = (ds['u100']**2 + ds['v100']**2)**0.5

In [ ]:
# compute daily mean
ds_daily = ds.resample(time='1D').mean('time')
ds_daily

### Create the map

We use cartopy package that, together with matplotlib, helps plot variables with different projections

All the available projections can be found [here](https://scitools.org.uk/cartopy/docs/v0.13/crs/projections.html#cartopy-projections)


In [ ]:
# set the desire countour levels
clevs = np.arange(0, 28, 4)
# create the lat lon grid from the data latitudes and longitudes
lons, lats = np.meshgrid(ds_daily.longitude.values, ds_daily.latitude.values)
# create the figure instance, the top level container of the figure
fig = plt.figure(figsize=(16, 11))
crs_latlon = ccrs.PlateCarree()
# generate the axes which have the projection that we will use
ax = plt.subplot(projection=ccrs.PlateCarree(central_longitude=0))
# set the limits of the domain relative to the projection
ax.set_extent([-12, 20, 36, 60], crs_latlon)
# filled contours of the magnitude of wind at 10mm for the first date
im=ax.contourf(lons, lats, ds_daily['ws100'].isel(time=0), clevs, transform=crs_latlon,
               cmap='YlGnBu', extend='max')
# uncomment the following line in you want to add contours to the shading
# ax.contour(lons, lats, ds_daily['ws100'].isel(time=0), colors='k', linewidths=0.5,
#           levels=clevs, transform=crs_latlon)
# generate colorbar and place it besides the plot
cbar = plt.colorbar(im, fraction=0.052, pad=0.04, shrink=0.8, aspect=15)
# tune the labels and ticks
cbar.ax.tick_params(labelsize=12)
cbar.set_label('m/s', labelpad=-40, y=-0.02, rotation=0, fontsize=15)
# add coastlines. resolution can be coarser (for instance, 50m)
ax.coastlines(resolution='10m', color='black', linewidth=1)
# add country borders
ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=.7)
# add gridlines, lat and lon and tune the labels
gl = ax.gridlines(crs=crs_latlon, draw_labels=True, linewidth=0.3, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 12, 'color': 'gray'}
gl.ylabel_style = {'size': 12, 'color': 'gray'}
# add figure title
ax.set_title('Mean Wind Speed at 100m - ' + str(ds_daily.time[0].values.astype('datetime64[D]')),
             fontsize=15)
# save figure with 300 dpi resolution
plt.savefig('MagU_100m_' + str(ds.time[0].values.astype('datetime64[D]'))+ '.jpg',
            dpi=300, bbox_inches='tight')